# Исследование надёжности заёмщиков

Кредитный отдел банка попросил разобраться, влияет ли на факт погашения кредит с срок семейное положение и количество детей. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования нужны при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
from pymystem3 import Mystem
m = Mystem()
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# посмотрим общую информацию
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# посмотрим основные статистические характеристики данных по каждому числовому признаку 
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

Рассмотрим полученную информацию подробнее.
Всего в таблице 12 столбцов:

 - children — количество детей в семье
 - days_employed — общий трудовой стаж в днях
 - dob_years — возраст клиента в годах
 - education — уровень образования клиента
 - education_id — идентификатор уровня образования
 - family_status — семейное положение
 - family_status_id — идентификатор семейного положения
 - gender — пол клиента
 - income_type — тип занятости
 - debt — имел ли задолженность по возврату кредитов
 - total_income — ежемесячный доход
 - purpose — цель получения кредита


В столбцах days_employed и total_income (общий трудовой стаж в днях и ежемесячный доход) есть пропущенные значения.

## Шаг 2. Предобработка данных

### Лемматизация

In [5]:
# Приведём значения из столбцов 'education' и 'family_status' к нижнему регистру
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()

In [6]:
# Посмотрим уникальные значение в столбце 'gender'
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Один XNA можно будет удалить, он не повлияется на результаты.

In [7]:
# Создаем новую таблицу, куда мы добавляем все строки, кроме строки, что в столбце gender есть значение XNA
df = df.loc[df['gender'] != 'XNA']

In [8]:
# Посмотрим уникальные значение в столбце 'purpose'
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Данных значений очень много, объединим данные в несколько подгрупп

In [9]:
# Создадим функцию, которая будет брать каждую строку и если в ней есть определенное слово
# то есть всё значение будет заменять на обредённый текст
def category_purpose(row):
    lem_purpose = m.lemmatize(row)
    if 'недвижимость' in lem_purpose:
        return 'Операции с недвижимостью'
    if 'образование' in lem_purpose:
        return 'Для образования'
    if 'свадьба' in lem_purpose:
        return 'Для проведения свадьбы'
    if 'автомобиль' in lem_purpose:
        return 'Покупка автомобиля'
    if 'жилье' in lem_purpose:
        return 'Ипотека'

# Заменим имеющиюся цели кредита на изменённые нами
df['purpose']=df['purpose'].apply(category_purpose)

In [10]:
# проверим нашу замену
df['purpose'].value_counts()

Операции с недвижимостью    6366
Ипотека                     4473
Покупка автомобиля          4315
Для образования             4022
Для проведения свадьбы      2348
Name: purpose, dtype: int64

**Вывод**

Такое большое разноообразия в одинаковых по смыслу ячейках произошел из-за возможности заполнять данные графы клиентам самостоятельно. Ведь одну и ту же цель можно описать множеством способов. Потому я привела их к единым целям:
 - "Для проведения свадьбы"
 - "Операции с недвижимостью"
 - "Покупка автомобиля"
 - "Для образования"
 - "Ипотека"

Ипотеку и операции с недвижимостью решила разбить на две разные группы, так как была формулировка "покупка коммерческой недвижимости", вовсе не обязательно, что где присутствовало слово "недвижимость", что это именно для собственного жилья, поэтому решила вывести в отдельную цель кредита.

Аналогично для образования. Всё приведено к нижнему регистру, чтобы не было дубликатов.

В данном массиве есть следующие словари:

В графах education и education_id:
Высшее образование - 0
Среднее образование - 1
Неоконченное высшее - 2
Начальное образование - 3
Учёная степень - 4


В графах family_status 	family_status_id:
Женат / замужем - 0
Гражданский брак - 1
Вдовец / вдова - 2
В разводе - 3
Не женат / не замужем - 4


в графе gender это:
F - женский пол
M - мужской пол

В графе debt это:
0 - не имел задолженности
1 - имел задолженность перед банком

###  Обработка дубликатов

In [11]:
# получение суммарного количества дубликатов в таблице df
df.duplicated().sum()

315

In [12]:
# удаление всех дубликатов из таблицы df с помощью метода
df=df.drop_duplicates().reset_index(drop=True)

In [13]:
# проверка на отсутствие
df.duplicated().sum()

0

**Вывод**

Дубликаты могли появиться вследствие сбоя в записи данных.
Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

### Изменение недопустимых значений

Также нам нужно привести данные в порядок в столбцах, содержащих информацию о количестве детей, общем трудовом стаже и возрасте клиента. Ведь данная информация не может быть минусовой (для всех трёх столбцов) и нулевой (для стобца с возрастом).

In [14]:
# Посмотрим уникальные значения столбика 'children'
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Отрицательное число детей невозможно. А количество детей в 20 маловероятно. Их нужно заменить.
Заменим по медиане.

In [15]:
df.loc[(df.children<0), 'children'] = df['children'].median()
df.loc[(df.children>10), 'children'] = df['children'].median()
# проверим исчезновение недопустимых значений
df['children'].unique()

array([1., 0., 3., 2., 4., 5.])

In [16]:
# Посмотрим сколько у нас клиентов, которых возраст меньше 18 лет
print('Количество клиентов, которые указали свой возраст меньше 18 лет -', len(df.loc[(df.dob_years < 18)]), 'человек')

Количество клиентов, которые указали свой возраст меньше 18 лет - 101 человек


In [17]:
df.loc[(df.dob_years < 18), 'dob_years'] = df['dob_years'].median()

In [18]:
# Выведем медиану столбца с трудовым стажем
print('Медиана трудового стажа', df['days_employed'].median(), 'дней')

Медиана трудового стажа -1203.307346482598 дней


In [19]:
# Заменим значения в стаже на модуль
df['days_employed'] = df['days_employed'].abs()

In [20]:
# выведем медиану по трудовому стажу для каждого возраста
df.groupby('dob_years')['days_employed'].median()

dob_years
19.0       724.492610
20.0       674.838979
21.0       618.733817
22.0       703.310078
23.0       690.204208
24.0       942.390603
25.0       919.199388
26.0      1083.658132
27.0      1166.212160
28.0      1141.705450
29.0      1315.453550
30.0      1420.586863
31.0      1308.901135
32.0      1446.622542
33.0      1426.415003
34.0      1615.910188
35.0      1613.494818
36.0      1799.520465
37.0      1816.713382
38.0      1817.194286
39.0      1891.388566
40.0      1728.936706
41.0      1864.657692
42.0      2210.575665
43.0      1869.019899
44.0      2084.330015
45.0      2254.431130
46.0      2100.473217
47.0      2203.078545
48.0      2429.674518
49.0      2560.317106
50.0      2626.986257
51.0      2846.080700
52.0      3395.639568
53.0      3650.007523
54.0      4026.541145
55.0      7043.471139
56.0      6850.179463
57.0    333953.491811
58.0    331245.976666
59.0    344372.337365
60.0    349414.537285
61.0    343984.931910
62.0    354981.718187
63.0    358482.295938


Все неадекватно большие числа (больше 12000 = 1200 / 250 р.д. = 48 рабочих лет) разделим на возраст клиента.
Клиента с возрастом 75 лет удалим из таблицы, т.к. это сильно не повлияет на результаты.

In [21]:
# Создадим функцию, где для каждой строки, где значение стажа больше 12000 дней
# Будет происходить деление стажа на возраст клиента
# Во всех остальных случаях будет возвращаться просто строка с изначальным стажем
def function(row):    
    if row['days_employed'] > 12000:
        return row['days_employed'] / row['dob_years']
    return row['days_employed']

# Заменим значения стажа по нашей функции
df['days_employed'] = df.apply(function, axis=1)

In [22]:
# Создаем новую таблицу, куда мы добавляем все строки, кроме строки, где возраст клиента равен 75
df = df.loc[df['dob_years'] != 75]

**Вывод**

Неадекватные значения детей и возраста были заменены по медиане.

Отрицательные значения стажа были заменены на индентичные, но со знаком плюс.

Огромные количество трудового стажа были заменены из предположения, что такие числа получились из-за технической ошибки умножения возраста клиента на его трудовой стаж.

Стаж свыше 12000 дней (48 рабочих лет) делился на возраст клиента.

Есть огромные проблемы с выгрузкой данных. Возможно они происходят из разных источников, где различаются единицы измерения. Нужно поставить задачу по выявлению проблемных данных. Например, таких как отрицательные значение детей или трудового стажа.

### Обработка пропусков

In [23]:
# суммарное количество пропусков в таблице df, выявленных методом isnull()
df.isnull().sum()

children               0
days_employed       1859
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        1859
purpose                0
dtype: int64

In [24]:
# Заменим пропущенные значения по столбцу 'total_income' на медиану в разрезе возраста и типа занятости
df['total_income'] = df['total_income']\
.fillna(df.groupby(['income_type', 'dob_years'])['total_income']\
.transform('median'))

# И пропущенные значения по столбцу 'days_employed' на медиану в разрезе возраста и типа занятости
df['days_employed'] = df['days_employed']\
.fillna(df.groupby(['income_type', 'dob_years'])['days_employed']\
.transform('median'))

In [25]:
# Остальные строки удалим, они никак не повлияются на наши вычисления
df.dropna(axis=0,inplace=True)

**Вывод**

Пропуски могли возникнуть из-за системной ошибки, или же просто люди при заполнении информации в банк решили не указывать свой заработок и/или трудовой стаж.
Заполнила пропуски в разрезе по возрасту, так как если бы мы брали медиану в целом, то было бы заполнено относительно общего обращения в банк всех людей, а средний возраст таких людей около 43 лет. У таких людей соотвественно выше стаж по сравнению, например, с 20-ти летними.

### Замена типа данных

In [26]:
#замена вещественного типа данных на целочисленный для столбцов days_employed и total_income
df['days_employed']=df['days_employed'].astype('int')
df['total_income']=df['total_income'].astype('int')

In [27]:
# проверим, поменялся ли тип данных
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21204 entries, 0 to 21208
Data columns (total 12 columns):
children            21204 non-null float64
days_employed       21204 non-null int64
dob_years           21204 non-null float64
education           21204 non-null object
education_id        21204 non-null int64
family_status       21204 non-null object
family_status_id    21204 non-null int64
gender              21204 non-null object
income_type         21204 non-null object
debt                21204 non-null int64
total_income        21204 non-null int64
purpose             21204 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


**Вывод**

Мы заменили тип данных на целочисленный в столбцах содержащих такую информацию как общий трудовой стаж и ежемесячный доход, так как это целочисленные значения. Нельзя указывать количество трудового стажа, как, например, 1,58 дня.

### Категоризация данных

In [28]:
# Создадим отдельную таблицу со столбцами 'children' и 'debt'
child_debt = df[['children','debt']]
# Посмотрим в разрезе по количеству детей
# Cколько клиентов с тем или иным количеством детей имел или не имел задолженность перед банком
# где 1 - имел задолженность, 0 - не имел.
child_debt.groupby('children')['debt'].value_counts()

children  debt
0.0       0       12939
          1        1071
1.0       0        4329
          1         444
2.0       0        1848
          1         194
3.0       0         302
          1          27
4.0       0          37
          1           4
5.0       0           9
Name: debt, dtype: int64

In [29]:
# Создадим отдельную таблицу со столбцами 'family_status' и 'debt'
family_status_debt = df[['family_status','debt']]
# Посмотрим в разрезе по семейному положению
# Cколько клиентов с тем или иным семейным положением имел или не имел задолженность перед банком
# где 1 - имел задолженность, 0 - не имел.
family_status_debt.groupby('family_status')['debt'].value_counts()

family_status          debt
в разводе              0        1108
                       1          85
вдовец / вдова         0         884
                       1          63
гражданский брак       0        3739
                       1         388
женат / замужем        0       11216
                       1         930
не женат / не замужем  0        2517
                       1         274
Name: debt, dtype: int64

In [30]:
# Создадим отдельную таблицу со столбцами 'total_income' и 'debt'
total_income_debt = df[['total_income','debt']]

def income_class(row):
        """
        Создадим классификацию по доходам
        Возвращает класс по доходам по значению доходов, используя правила:
        - 'Низкие доходы' при значении income < 20 000
        - 'Средне низкие доходы' при значениии более 20 000 и менее 50 000, включая 50 000
        - 'Средние доходы' при значениии более 50 000 и менее 100 000, включая 100 000
        - 'Средне высокие доходы' при значениии более 100 000 и менее 150 000, включая 150 000
        - 'Высокие доходы' при значениии более 150 000 и менее 250 000, включая 250 000
        - 'Безработные' во всех остальных случаях
        """

        if row['total_income'] < 20000:
                return 'Низкие доходы'
        if row['total_income'] <=  50000:
                return 'Средне низкие доходы'
        if row['total_income'] <=  100000:
                return 'Средние доходы' 
        if row['total_income'] <=  150000:
                return 'Средне высокие доходы'
        if row['total_income'] <=  250000:
                return 'Высокие доходы'

total_income_debt['income_class'] = total_income_debt.apply(income_class, axis=1)

# Посмотрим в разрезе по доходам
# Cколько клиентов с тем или иным заработком имел или не имел задолженность перед банком
# где 1 - имел задолженность, 0 - не имел.
total_income_debt.groupby('income_class')['debt'].value_counts()

income_class           debt
Высокие доходы         0       6443
                       1        577
Средне высокие доходы  0       6286
                       1        615
Средне низкие доходы   0        349
                       1         23
Средние доходы         0       3765
                       1        331
Name: debt, dtype: int64

In [31]:
# Создадим отдельную таблицу со столбцами 'purpose' и 'debt'
purpose_debt = df[['purpose','debt']]
# Посмотрим в разрезе по целям кредита
# Cколько клиентов с той или иной целью кредита имел или не имел задолженность перед банком
# где 1 - имел задолженность, 0 - не имел.
purpose_debt.groupby('purpose')['debt'].value_counts()

purpose                   debt
Для образования           0       3593
                          1        370
Для проведения свадьбы    0       2119
                          1        186
Ипотека                   0       4099
                          1        308
Операции с недвижимостью  0       5784
                          1        474
Покупка автомобиля        0       3869
                          1        402
Name: debt, dtype: int64

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок

In [32]:
# Выведем таблицу, с количеством детей, где указано сколько людей не имело задолженностей перед банком
# И сколько имело задолженности, и сразу выведем в процентах количество должников
df.groupby('children').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                 
          count   sum <lambda_0>
children                        
0.0       14010  1071      7.64%
1.0        4773   444       9.3%
2.0        2042   194       9.5%
3.0         329    27      8.21%
4.0          41     4      9.76%
5.0           9     0       0.0%

**Вывод**

Самые надежные клиенты это люди имеющие пятерых детей.
Среди них 0% должников.
Но брать этот вывод как константу не стоит, ведь в данной выборке участвовало всего 9 клиентов.

Так что скорее всего **самые надежные** клиенты - это люди без детей, имеют меньше задолженностей перед банком. Всего **7,64% людей** не возвращают кредиты во время.

А **самые безответственные** - это люди, имеющие в семье 4х детей: **9,76% людей** из данной выборки не возвращают кредиты в срок.

Семьи с количеством детей 1, 2 и 3 имеют следующие показатели задолженностей перед банком: 9,3%, 9,5% и 8,21%.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
# Выведем таблицу, с семейным статусом, где указано сколько людей в том или ином статусе
# не имело задолженностей перед банком и сколько имело задолженности, и сразу выведем в процентах количество должников
df.groupby('family_status').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                       count  sum <lambda_0>
family_status                               
в разводе               1193   85      7.12%
вдовец / вдова           947   63      6.65%
гражданский брак        4127  388       9.4%
женат / замужем        12146  930      7.66%
не женат / не замужем   2791  274      9.82%

**Вывод**

**Самые надежные** клиенты это люди, находящиеся в статусе вдов / вдовцов. Среди них **6,65% дожников**.

А **самые безответственные** - это люди, не вступавшие в брак: **9,82% людей** из данной выборки не возвращают кредиты в срок.

Средние показатели показывают люди, находящиеся в разводе или же состоящих в статусе "женат / замужем" - 7,12% и 7,66% должников в данной выборке.

Люди, находящиеся в гражданском браке приближены к самым безответсвенным: 9,4% должников.

Возможно это связано с законодательством РФ, о том, что долги семейных людей по закону выплачиваются в равной степени мужем и женой. В гражанском браке же, каждый несёт отвественность сам за себя.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
# Выведем таблицу, с доходами, где указано сколько людей с тем или иным заработком
# не имело задолженностей перед банком и сколько имело задолженности, и сразу выведем в процентах количество должников
total_income_debt.groupby('income_class').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                      count  sum <lambda_0>
income_class                               
Высокие доходы         7020  577      8.22%
Средне высокие доходы  6901  615      8.91%
Средне низкие доходы    372   23      6.18%
Средние доходы         4096  331      8.08%

**Вывод**

**Самые надежные** клиенты это люди, имеющие доход от 20 до 50 т.р.: **6.18% клиентов** не вернувших вовремя кредит.

А **самые безответсвенные** - это люди, имеющие доход от 100 до 150 т.р.: **8,91% должников**

Средние показатели показывают люди, имеющие доход от 50 до 100 т.р. и высокие доходы от 150 до 250 т.р. - 8,08% и 8,22% должников соответственно.

Возможно это связано с количеством свободных денег: люди, имеющие больший заработок, активнее возьмут кредит, т.к. уверены, что могут его вернуть. И также для них просрочка не проблема, могут просто забыть заплатить вовремя.
А люди с меньшими доходами с опаской относятся к кредитам, так как не уверены, что смогут выплатить его вовремя.

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
# Выведем таблицу, с целями кредита, где указано сколько людей с той или иной целью
# не имело задолженностей перед банком и сколько имело задолженности, и сразу выведем в процентах количество должников
df.groupby('purpose').agg({'debt':['count', 'sum', lambda x: str(round(x.mean()*100,2)) +'%' ]})

debt                
                         count  sum <lambda_0>
purpose                                       
Для образования           3963  370      9.34%
Для проведения свадьбы    2305  186      8.07%
Ипотека                   4407  308      6.99%
Операции с недвижимостью  6258  474      7.57%
Покупка автомобиля        4271  402      9.41%

**Вывод**

**Самые надежные** клиенты это люди, берущие кредит для Ипотеки : **6,99%** задолженностей было в данной выборке.

А **самые безответсвенные** - это люди, берущие кредит для покупки автомобиля: **9,41%** должников среди таких клиентов.

Средние показатели показывают люди, берущие кредит для операция с недвижимостью и проведения свадьбы. - 7,57% и 8,07% должников соответственно.

Люди, берущие кредит для получения образования к самым безответсвенным: 9,34% должников.

Возможно это связано с тем, что людям не хватает их заработков и для их повышения они идут учится или же покупают автомобиль, чтобы с помощью данных инструментов больше зарабатывать.

Люди берущие ипотеку, улучшают свою кредитную историю, чтобы получить более выгодные условия для кредита. Ведь их кредит является долгосрочным.

## Шаг 4. Общий вывод

В целом процент должников в массе людей имеет примерный показатель ~8%.

Самыми отвественные клиентами являют люди заработывающие от 20 до 50 тысяч, они показывали самое низкое количество должников - 6,18%.

А самые безответственные клиенты это люди, состоящих в статусе "не женат / не замужем" - среди них 9,82% должников.

Если не брать в расчет наибольшие и наименьшие проценты должников среди тех или иных групп населения.
Самыми надёжными средними являются люди, находящиеся в официальном браке, не имеющие детей, имеющие доход свыше 50 тысяч рублей, берущих кредит для ипотеки, свадьбы или же операций с недвижимостью.